In [1]:
pip install opencv-python-headless


Note: you may need to restart the kernel to use updated packages.


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np

class AugmentedDataGenerator(ImageDataGenerator):
    def __init__(self, contrast_range=None, color_jitter_range=None, noise_range=None, *args, **kwargs):
        self.contrast_range = contrast_range
        self.color_jitter_range = color_jitter_range
        self.noise_range = noise_range
        super(AugmentedDataGenerator, self).__init__(*args, **kwargs)
        
    def __next__(self):
        batch = super(AugmentedDataGenerator, self).__next__()
        for i, image in enumerate(batch[0]):
            # random brightness
            if self.brightness_range:
                if np.random.random() < 0.5:
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
                    brightness_factor = np.random.uniform(*self.brightness_range)
                    image[:, :, 2] = image[:, :, 2] * brightness_factor
                    image = cv2.cvtColor(image, cv2.COLOR_HSV2RGB)

            # random contrast
            if self.contrast_range:
                if np.random.random() < 0.5:
                    alpha = np.random.uniform(*self.contrast_range)
                    image = cv2.convertScaleAbs(image, alpha=alpha)

            # random color jitter
            if self.color_jitter_range:
                if np.random.random() < 0.5:
                    noise = np.random.randint(-self.color_jitter_range, self.color_jitter_range, image.shape)
                    image = cv2.add(image, noise)

            # random noise
            if self.noise_range:
                if np.random.random() < 0.5:
                    noise = np.random.normal(0, self.noise_range, image.shape)
                    image = cv2.add(image, noise)

            batch[0][i] = image

        return batch



In [3]:
pip install keras-tuner


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [6]:
import tensorflow as tf

# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
    print('GPU is available')
else:
    print('GPU is not available')


GPU is available


In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras_tuner.tuners import RandomSearch

# load train dataset
train_df = pd.read_csv('/Users/tim/Desktop/postgraduate/semester 2/machine learning 2/assessment_project/machine-learning-in-science-ii-2023/training_norm.csv')
train_images = []
for image_id in train_df['image_id']:
    image = cv2.imread(f'/Users/tim/Desktop/postgraduate/semester 2/machine learning 2/assessment_project/machine-learning-in-science-ii-2023/training_data/training_data/{image_id}.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    train_images.append(image)
train_images = np.array(train_images)
train_angles = np.array(train_df['angle'])
train_speeds = np.array(train_df['speed'])

# load test dataset
test_images = []
for i in range(1, 1021):
    image = cv2.imread(f'/Users/tim/Desktop/postgraduate/semester 2/machine learning 2/assessment_project/machine-learning-in-science-ii-2023/test_data/test_data/{i}.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    test_images.append(image)
test_images = np.array(test_images)
test_ids = np.arange(1, 1021)

# split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_df[['angle', 'speed']].values, test_size=0.2, random_state=42)

# define the early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# image augmentation
train_datagen = AugmentedDataGenerator(
    brightness_range=[0.5, 1.5],
    contrast_range=[0.8, 1.2],
    color_jitter_range=20,
    noise_range=10
)
train_datagen.fit(X_train)
val_datagen = AugmentedDataGenerator()
val_datagen.fit(X_val)




In [11]:
# define the model
def build_model(hp):
    # input layer
    input_layer = Input(shape=(240, 320, 3))

    # convolutional layers
    num_conv_layers = hp.Int('num_conv_layers', min_value=2, max_value=5, step=1)
    conv_layer_filters = [hp.Choice(f'conv_layer_{i}_filters', values=[32, 64, 128, 256], ordered=False, default=64) for i in range(num_conv_layers)]
    conv_layer_kernel_sizes = [hp.Choice(f'conv_layer_{i}_kernel_sizes', values=[3, 5, 7], ordered=False, default=3) for i in range(num_conv_layers)]
    conv_layer_activations = [hp.Choice(f'conv_layer_{i}_activations', values=['relu', 'elu'], default='relu') for i in range(num_conv_layers)]
    x = input_layer
    for i in range(num_conv_layers):
        x = Conv2D(filters=conv_layer_filters[i], kernel_size=conv_layer_kernel_sizes[i], activation=conv_layer_activations[i], padding='same')(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

    # fully connected layers
    x = Flatten()(x)
    num_dense_layers = hp.Int('num_dense_layers', min_value=1, max_value=3, step=1)
    for i in range(num_dense_layers):
        dense_layer_units = hp.Int(f'dense_layer_{i}_units', min_value=32, max_value=512, step=32)
        dense_layer_activation = hp.Choice(f'dense_layer_{i}_activation', values=['relu', 'elu'], default='relu')
        x = Dense(units=dense_layer_units, activation=dense_layer_activation)(x)
        dropout_rate = hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1, default=0.25)
        x = Dropout(rate=dropout_rate)(x)

    # output layer
    angle_output = Dense(units=1, name='angle_output')(x)
    speed_output = Dense(units=1, name='speed_output')(x)
    model = Model(inputs=input_layer, outputs=[angle_output, speed_output])

    # compile the model(with Mac chip)
    optimizer = Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log'))
    loss = {'angle_output': 'mean_squared_error', 'speed_output': 'mean_squared_error'}
    metrics = {'angle_output': 'mae', 'speed_output': 'mae'}
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

# define the tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    directory='output',
    project_name='steering_angle_regression'
)



INFO:tensorflow:Reloading Tuner from output/steering_angle_regression/tuner0.json


In [ ]:
# search for the best hyperparameters
tuner.search(train_datagen.flow(X_train, y_train, batch_size=32), epochs=50, validation_data=val_datagen.flow(X_val, y_val, batch_size=32), callbacks=[early_stop])

# get the best model
best_model = tuner.get_best_models(num_models=1)[0]
# save the best model as an .h5 file
best_model.save('aug_with_epoch_change_model.h5')

# train the model on the entire training dataset
history = best_model.fit(train_datagen.flow(train_images, (train_angles, train_speeds), batch_size=32), epochs=100, validation_data=val_datagen.flow(X_val, y_val, batch_size=32), callbacks=[early_stop])

# make predictions on the test set
test_predictions = best_model.predict(test_images)

# get the steering angle and speed predictions separately
test_steering_predictions = test_predictions[0]
test_speed_predictions = test_predictions[1]
# speed adjustment
if np.min(test_speed_predictions) < 0:
    test_speed_predictions = test_speed_predictions - np.min(test_speed_predictions) #if there is negative num in spd column
    max_val = np.max(test_speed_predictions)      
    normalized_arr_spd = test_speed_predictions / max_val
    test_speed_predictions = np.where(normalized_arr_spd >= 0.5, 1, 0)
else:
    max_val = np.max(test_speed_predictions)      
    normalized_arr_spd = test_speed_predictions / max_val
    test_speed_predictions = np.where(normalized_arr_spd >= 0.5, 1, 0)
submission_df = pd.DataFrame({
    'image_id': test_ids.flatten(),
    'angle': test_steering_predictions.flatten(),
    'speed': test_speed_predictions.flatten()
})


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
5                 |4                 |num_conv_layers
256               |128               |conv_layer_0_filters
128               |128               |conv_layer_1_filters
7                 |5                 |conv_layer_0_kernel_sizes
7                 |5                 |conv_layer_1_kernel_sizes
relu              |elu               |conv_layer_0_activations
relu              |elu               |conv_layer_1_activations
1                 |1                 |num_dense_layers
96                |160               |dense_layer_0_units
relu              |elu               |dense_layer_0_activation
0.1               |0.2               |dropout_0
0.0090213         |0.0022124         |learning_rate
32                |64                |conv_layer_2_filters
128               |64                |conv_layer_3_filters
3                 |3                 |conv_layer_2_kernel_sizes
5                 |3                

Epoch 1/50
  6/345 [..............................] - ETA: 11:55 - loss: 824073519104.0000 - angle_output_loss: 719611559936.0000 - speed_output_loss: 104461950976.0000 - angle_output_mae: 376170.0000 - speed_output_mae: 141014.8906 WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.6294s vs `on_train_batch_end` time: 1.4875s). Check your callbacks.


123/345 [=========>....................] - ETA: 7:37 - loss: 40200372224.0000 - angle_output_loss: 35104591872.0000 - speed_output_loss: 5095779328.0000 - angle_output_mae: 18525.4160 - speed_output_mae: 6946.9092

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG14XFamilyCommandBuffer: 0x322151e60>
    label = <none> 
    device = <AGXG14SDevice: 0x2ca1fa400>
        name = Apple M2 Pro 
    commandQueue = <AGXG14XFamilyCommandQueue: 0x13a884400>
        label = <none> 
        device = <AGXG14SDevice: 0x2ca1fa400>
            name = Apple M2 Pro 
    retainedReferences = 1


145/345 [===========>..................] - ETA: 6:50 - loss: 34101004288.0000 - angle_output_loss: 29778378752.0000 - speed_output_loss: 4322626560.0000 - angle_output_mae: 15714.8496 - speed_output_mae: 5893.0532

...

151/345 [============>.................] - ETA: 6:33 - loss: 0.9957 - angle_output_loss: 0.3297 - speed_output_loss: 0.6660 - angle_output_mae: 0.5219 - speed_output_mae: 0.7455

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG14XFamilyCommandBuffer: 0x320438620>
    label = <none> 
    device = <AGXG14SDevice: 0x2ca1fa400>
        name = Apple M2 Pro 
    commandQueue = <AGXG14XFamilyCommandQueue: 0x13a884400>
        label = <none> 
        device = <AGXG14SDevice: 0x2ca1fa400>
            name = Apple M2 Pro 
    retainedReferences = 1


234/345 [===================>..........] - ETA: 3:45 - loss: 0.9870 - angle_output_loss: 0.3262 - speed_output_loss: 0.6608 - angle_output_mae: 0.5193 - speed_output_mae: 0.7421

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG14XFamilyCommandBuffer: 0x327575ce0>
    label = <none> 
    device = <AGXG14SDevice: 0x2ca1fa400>
        name = Apple M2 Pro 
    commandQueue = <AGXG14XFamilyCommandQueue: 0x13a884400>
        label = <none> 
        device = <AGXG14SDevice: 0x2ca1fa400>
            name = Apple M2 Pro 
    retainedReferences = 1


251/345 [====================>.........] - ETA: 3:21 - loss: 0.9929 - angle_output_loss: 0.3260 - speed_output_loss: 0.6669 - angle_output_mae: 0.5189 - speed_output_mae: 0.7443

In [ ]:
def custom_generator(images, angles, speeds, batch_size, datagen):
    num_samples = len(images)
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_images = images[offset:offset+batch_size]
            batch_angles = angles[offset:offset+batch_size]
            batch_speeds = speeds[offset:offset+batch_size]

            augmented_images = []
            for img in batch_images:
                augmented_img = datagen.random_transform(img)
                augmented_images.append(augmented_img)

            yield np.array(augmented_images), [np.array(batch_angles), np.array(batch_speeds)]


In [ ]:
train_generator = custom_generator(train_images, train_angles, train_speeds, 32, train_datagen)
val_generator = custom_generator(X_val, y_val[:, 0], y_val[:, 1], 32, val_datagen)
history = best_model.fit(train_generator, steps_per_epoch=len(train_images) // 32, epochs=100, validation_data=val_generator, validation_steps=len(X_val) // 32, callbacks=[early_stop])

# make predictions on the test set
test_predictions = best_model.predict(test_images)

# get the steering angle and speed predictions separately
test_steering_predictions = test_predictions[0]
test_speed_predictions = test_predictions[1]

# speed adjustment
if np.min(test_speed_predictions) < 0:
    test_speed_predictions = test_speed_predictions - np.min(test_speed_predictions) #if there is negative num in spd column
    max_val = np.max(test_speed_predictions)      
    normalized_arr_spd = test_speed_predictions / max_val
    test_speed_predictions = np.where(normalized_arr_spd >= 0.5, 1, 0)
else:
    max_val = np.max(test_speed_predictions)      
    normalized_arr_spd = test_speed_predictions / max_val
    test_speed_predictions = np.where(normalized_arr_spd >= 0.5, 1, 0)
submission_df = pd.DataFrame({
    'image_id': test_ids.flatten(),
    'angle': test_steering_predictions.flatten(),
    'speed': test_speed_predictions.flatten()
})
submission_df.to_csv('aug_with_change_submission.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

# generate augmented images
augmented_images = train_datagen.flow(X_train, y_train, batch_size=32)
augmented_images = next(augmented_images)

# plot the augmented images
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(12, 12))
for i, ax in enumerate(axes.flat):
    ax.imshow(augmented_images[0][i])
plt.show()


In [2]:
import pandas as pd

# Create a dictionary with the augmentation techniques
data = {
    'Augmentation Technique': ['Random Brightness', 'Random Contrast', 'Random Color Jitter', 'Random Noise'],
    'Parameter Range': [
        '[0.5, 1.5]',
        '[0.8, 1.2]',
        '-20 to 20',
        'Normal distribution with std dev of 10'
    ]
}

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)


  Augmentation Technique                         Parameter Range
0      Random Brightness                              [0.5, 1.5]
1        Random Contrast                              [0.8, 1.2]
2    Random Color Jitter                               -20 to 20
3           Random Noise  Normal distribution with std dev of 10
